# Исследование надёжности заёмщиков <a id="1"></a> 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Задача:** На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

**Содержание проекта**
1. [Описание проекта](#1)
2. [Предобработка данных](#2)
    * [Обработка пропусков](#2.1)
    * [Замена типа данных](#2.2)
    * [Обработка дубликатов](#2.3)
    * [Лемматизация](#2.4)
    * [Категоризация данных](#2.5)
3. [Ответы на вопросы](#3)
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#3.1)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#3.2)  
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#3.3)
    * [Как разные цели кредита влияют на его возврат в срок?](#3.4)
4. [Общие выводы](#4)

## Предобработка данных <a id="2"></a> 

In [1]:
import pandas as pd
try:
    data=pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') #ссылка, работающая везде, где есть интернет
except:
    data=pd.read_csv('/yandex/data.csv') #локальный путь к файлу
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
#Посмотрим процентное кол-во пропусков в столбцах
pd.DataFrame(round(data.isna().mean()*100,0)).style.background_gradient('coolwarm')

,0
children,0.000000
days_employed,10.000000
dob_years,0.000000
education,0.000000
education_id,0.000000
family_status,0.000000
family_status_id,0.000000
gender,0.000000
income_type,0.000000
debt,0.000000


**Вывод**


В датафрейме есть пропуски в столбцах 'days_emloyed' и 'total_income', которые составляют примерно 10% от общего кол-ва строк. Также в столбце 'days_employed' встречаются артефакты. На первый взгляд, можно сказать, что данные из этих столбцов не нужны для проведения исследования и их можно просто удалить, но 10% - это большой объем инйформации и его удаление может привезти к неточности результата и потери важной информации. Скорее всего, данные пропущены из-за различных кредитных программ банка (есть такие программы кредитования, в которых не требуется информация о стаже и ежемесячном доходе - для пенсионеров, зарплатных клиентов банка, постоянных клиентов с крупными оборотами по счету и т.д.). Либо же данные пропуски связаны с техническими сбоями при копировании данных и тд. Nan - это пропуск типа float. Также можем точно сказать, что пропущены количественные переменные. Поэтому обрабатывать пропуски следует методами для количественных переменных.

### Обработка пропусков  <a id="2.1"></a> 

Пропуски бывают трёх видов: 1)Полностью случайные; 2)Случайные; 3)Неслучайные. 
    Изучив таблицу, мы можем сделать вывод, что в обоих случаях пропуски неслучайные, т.к. допущены нарочно, их нельзя с легкостью восстановить по данным других столбцов и они зависят от других значений, в том числе и от значений собственного столбца.

Начнем с обработки пропусков в столбце 'total_income'. 

In [3]:
#выведем на экран все уникальные значения столбца 'income_type' а также посмотрим их кол-во
print(data['income_type'].value_counts()) 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Мы видим, что количество предпринимателей, безработных, студентов и находящихся в декрете ничтожно мало по сравнению с другими типами занятости, сравнивать их между с собой будет неккоректно, поэтому присоединим данные типы к категории 'госслужащий' и выведем на экран результат.

In [4]:
data.loc[data['income_type']=='студент', 'income_type']='госслужащий'
data.loc[data['income_type']=='в декрете', 'income_type']='госслужащий'
data.loc[data['income_type']=='предприниматель', 'income_type']='госслужащий'
data.loc[data['income_type']=='безработный', 'income_type']='госслужащий'
print(data['income_type'].value_counts()) 

сотрудник      11119
компаньон       5085
пенсионер       3856
госслужащий     1465
Name: income_type, dtype: int64


 Мы уже выяснили, что пропуски являются количественными переменными, поэтому мы будем использовать один из методов mean() или median() для заполнения таких пропусков. Но вначале, определимся на основе каких столбцов мы будем осуществлять группировку и корректировку по медиане или среднему 

In [5]:
data.corr()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,1.000000,-0.138354,-0.174358,-0.012695,-0.091284,0.018470,0.018220
days_employed,-0.138354,1.000000,0.582643,0.080565,0.005726,-0.047110,-0.136648
dob_years,-0.174358,0.582643,1.000000,0.067467,-0.069988,-0.069620,-0.052911
education_id,-0.012695,0.080565,0.067467,1.000000,0.007876,0.052906,-0.178885
family_status_id,-0.091284,0.005726,-0.069988,0.007876,1.000000,0.020611,-0.009147
debt,0.018470,-0.047110,-0.069620,0.052906,0.020611,1.000000,-0.012475
total_income,0.018220,-0.136648,-0.052911,-0.178885,-0.009147,-0.012475,1.000000


По коэффициенту корреляции столбец days_employed лучше всего заполнить на основе столбца dob_years. А на столбец total_income больше всех, но при этом не значительно влияет столбец children (всего 0.018220). Данная таблица конечно же не учитывает столбцы со строковыми данными, поэтому можем предположить, что тип занятости или другими словами "должность" больше влияет на величину дохода, чем дети или к примеру семейное положение.

Заполним пропуски в столце 'total_income' медианными значениями (среднее не подойдет,т.к слишком отдалено от реалий), но с корректировкой по типу занятости (для каждого типа найдем свою медиану и заполним данные).

In [6]:
#запишем функцию, которая заменит наши пропуски на медиану уровня дохода для каждого типа занятости. 
#В качестве переменных пропишем наш датасет, столбец с доходом и вид заработка.
def median_total_income(df, value, category):
    #для каждого типа занятости в списке уникальных значений
    for type_unique in df[category].unique(): 
        #для каждого значения с пропуском и равному определенному типу занятости
        df.loc[(df[value].isna())&(df[category] ==type_unique), value] = \
        df.loc[df[category] ==type_unique, value].median()   
        #присваиваем значение медианы по данному типу занятости
    return df #возвращаем весь заполненный датасет

data = median_total_income(data, 'total_income', 'income_type')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [7]:
data['total_income'].isna().sum()
#проверим, что все пропуски в столбце 'total_income' заполнены. Отлично.

0

Столбец 'days_employed' содержит артефакты. Скорее всего при вводе данных указывали "-", не имея ввиду отрицательное число. Приведем к модулю значения столбца.

In [8]:
data['days_employed']=abs(data['days_employed'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 Если посмотреть на значения столбца, можно заметить, что данные отражены явно некорректно. Если стаж выражен в днях, то в строке 4, 18, 24 и тд  стаж составляет более 1000 лет. Скорее всего в этих строках стаж отражен не в днях, а в часах или в минутах. Попробуем это исправить. Скорее всего в столбце отражалась дата трудоустройства, но теперь данные сбились. Попробуем восстановить данные. Мы знаем, что у нас есть распространенный формат даты и времени - unix time. Он хранит значение в секундах. В нашем столбце данные хранятся в днях - переведем для начала в секунды.

In [9]:
#изучим столбец более внимательно, посмотрим максимальное и минимальное значение, а также среднее:
print(data['days_employed'].max()) 
print(data['days_employed'].min())
print(data['days_employed'].mean())

401755.40047533
24.14163324048118
66914.72890682236


 Исходя из данных выше, можно сказать, что минимальное значение выражено все-таки в днях, а вот максимальное значение в часах (401755/24 = 16379, а уже 16379/365 = 45 лет - это уже более похоже на реальный стаж). Давайте будем считать, что стаж с 18 до 65 лет - это максимум 47 лет. Добавим еще 3 года  среднем, т.к. некоторые могут работать и после выхода на пенсию. Итого - 50 лет. Т.е значения выше 50*365 = 18250  будут считаться выраженными не в днях, а  часах, и будут подвергнуты корректировке. Кстати, если считать, что unix time ведет отсчет с 1970 года, то на данный момент стаж не может превышать 50 лет. 

In [10]:
data['days_employed']=data['days_employed'].apply(lambda x: x if x<=18250 else x/24)
print(data['days_employed'].head(10))
print(data['days_employed'].max())

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
Name: days_employed, dtype: float64
17615.563265627912


In [11]:
#теперь весь стобец приведем к секундам
data['days_employed']=data['days_employed'].apply(lambda x: x*86400)


 Сейчас следует заполнить пропуски, т.к потом, когда мы переведем данные в формат даты, мы уже не сможем заполнить данные медианой по группам.
Мы уже выяснили, что данный столбец наиболее зависим по корреляции от столбца 'dob_years'. Разделим столбец 'dob_years' на 5 возрастных групп: 

In [12]:
data['group_year'] = pd.qcut(data['dob_years'], 5) 

In [13]:
data['group_year']

0        (39.0, 47.0]
1        (31.0, 39.0]
2        (31.0, 39.0]
3        (31.0, 39.0]
4        (47.0, 56.0]
             ...     
21520    (39.0, 47.0]
21521    (56.0, 75.0]
21522    (31.0, 39.0]
21523    (31.0, 39.0]
21524    (39.0, 47.0]
Name: group_year, Length: 21525, dtype: category
Categories (5, interval[float64]): [(-0.001, 31.0] < (31.0, 39.0] < (39.0, 47.0] < (47.0, 56.0] < (56.0, 75.0]]

In [14]:
#функция подсчета медианы первой возрастной группы
def median1(age):
    sorted_data = data[data['dob_years']<=31]
    median_total_income= sorted_data['days_employed'].median()
    return median_total_income

In [15]:
#функция подсчета медианы второй возрастной группы
def median2(age):
    sorted_data = data.loc[(data['dob_years']>31)&(data['dob_years']<=39)]
    median_total_income= sorted_data['days_employed'].median()
    return median_total_income

In [16]:
#функция подсчета медианы третьей возрастной группы
def median3(age):
    sorted_data = data.loc[(data['dob_years']>39)&(data['dob_years']<=47)]
    median_total_income= sorted_data['days_employed'].median()
    return median_total_income

In [17]:
#функция подсчета медианы четвертой возрастной группы
def median4(age):
    sorted_data = data.loc[(data['dob_years']>47)&(data['dob_years']<=56)]
    median_total_income= sorted_data['days_employed'].median()
    return median_total_income

In [18]:
#функция подсчета медианы пятой возрастной группы
def median5(age):
    sorted_data = data[data['dob_years']>56]
    median_total_income= sorted_data['days_employed'].median()
    return median_total_income

In [19]:
#заполним пропуски в столбце 'days_employed' медианными значениями 
data.loc[(data['dob_years']<=31)&(data['days_employed'].isna()),'days_employed']=median1(data)
data.loc[(data['dob_years']>31)&(data['dob_years']<=39)&(data['days_employed'].isna()),'days_employed']=median2(data)
data.loc[(data['dob_years']>39)&(data['dob_years']<=47)&(data['days_employed'].isna()),'days_employed']=median3(data)
data.loc[(data['dob_years']>47)&(data['dob_years']<=56)&(data['days_employed'].isna()),'days_employed']=median4(data)
data.loc[(data['dob_years']>56)&(data['days_employed'].isna()),'days_employed']=median5(data)

In [20]:
data['days_employed'].isna().sum()

0

Пропусков нет, теперь можем перевести весь столбец в формат даты. 

In [21]:
data['days_employed'].astype('int') #приведем столбец к формату int, чтобы далее корректно привести к формату datetime

0         729014949
1         347743044
2         485863713
3         356378158
4        1224957859
            ...    
21520     391332959
21521    1238174654
21522     182593171
21523     268918419
21524     171461455
Name: days_employed, Length: 21525, dtype: int64

In [22]:
import time
time.strftime("%d.%m.%Y", time.localtime(729014949)) 
#таким методом мы можем привести весь столбец к формату datetime,
#воспользуемся для удобства lambda функцией.


'06.02.1993'

In [23]:
from datetime import datetime, timedelta
data['days_employed']=data['days_employed'].apply(lambda x: time.strftime("%d.%m.%Y", time.localtime(x)))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_year
0,1,06.02.1993,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"(39.0, 47.0]"
1,1,07.01.1981,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"(31.0, 39.0]"
2,0,25.05.1985,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"(31.0, 39.0]"
3,3,17.04.1981,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"(31.0, 39.0]"
4,0,25.10.2008,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"(47.0, 56.0]"


Теперь исходные данные восстановлены. Но если взглянуть на столбец 'dob_years' к примеру в строке 3, возраст равен 32 годам, т.е у данного человека стаж начинается с двухлетнего возраста, аналогично и по многим другим строкам. Получается, что стаж больше возраста или равен ему. Поэтому отсюда можно сделать вывод, что столбец все-таки с неккоректными данными, и в дальнейшем в исследовании мы им пользоваться не будем.

In [24]:
data.sample(10) # посмотрим рандомно несколько строк, чтобы исключить наличие скрытых пропусков 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_year
14735,0,21.03.1973,40,Среднее,1,гражданский брак,1,F,компаньон,0,203982.682943,сыграть свадьбу,"(39.0, 47.0]"
10855,0,03.11.1990,56,среднее,1,Не женат / не замужем,4,F,госслужащий,0,249879.759506,операции со своей недвижимостью,"(47.0, 56.0]"
8517,1,31.05.1978,33,высшее,0,гражданский брак,1,F,сотрудник,0,138839.875210,свадьба,"(31.0, 39.0]"
3239,0,07.12.1972,35,среднее,1,Не женат / не замужем,4,F,компаньон,0,83694.393699,заняться высшим образованием,"(31.0, 39.0]"
2639,0,28.08.1976,46,среднее,1,женат / замужем,0,M,компаньон,1,116896.346186,профильное образование,"(39.0, 47.0]"
973,0,05.10.1978,49,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,операции с жильем,"(47.0, 56.0]"
16258,1,26.10.1976,47,среднее,1,гражданский брак,1,M,компаньон,0,349963.437017,на проведение свадьбы,"(39.0, 47.0]"
1492,0,31.03.2014,69,высшее,0,женат / замужем,0,M,пенсионер,0,104979.350488,получение дополнительного образования,"(56.0, 75.0]"
8842,0,05.10.1978,51,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,недвижимость,"(47.0, 56.0]"
18812,0,28.12.1973,52,среднее,1,женат / замужем,0,F,сотрудник,0,482612.934778,покупка жилой недвижимости,"(47.0, 56.0]"


Как мы видим, скрытых пропусков в таблице нет, но есть "аномалии", такие как значение -1 и 20 в столбце 'children'.Обработаем их. 

In [25]:
data=data.loc[(data.loc[:,'children']!=20)&(data.loc[:,'children']!=-1)] 
# мы удалили строки с данными аномалиями, т.к их меньше 1% от общего кол-ва и на исследование это не повлияет

**Вывод**

Пропуски в столбцах 'total_income' и 'days_employed' обработаны. Также мы привели к модулю столбец 'days_employed', тем самым избавившись от артефактов. И мы даже смогли восстановить данные из того столбца и привести их к формату даты. Правда даже после восстановления, данные являются неккоректными и их дальнейшее использование нецелесообразно.

### Замена типа данных  <a id="2.2"></a> 

Поменяем тип данных в столбцах 'total_income'на целочисленный (int). Для этого воспользуемся методом astype(), который переведет тип данных в любой, который мы укажем в аргументе. Существует еще метод to_numeric(), но он переводит в тип float (вещественные числа), и в данном случае он нам не подходит.

In [26]:
data['total_income']=data['total_income'].astype(int)
#Проверим изменения в даатфрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21402 non-null  int64   
 1   days_employed     21402 non-null  object  
 2   dob_years         21402 non-null  int64   
 3   education         21402 non-null  object  
 4   education_id      21402 non-null  int64   
 5   family_status     21402 non-null  object  
 6   family_status_id  21402 non-null  int64   
 7   gender            21402 non-null  object  
 8   income_type       21402 non-null  object  
 9   debt              21402 non-null  int64   
 10  total_income      21402 non-null  int64   
 11  purpose           21402 non-null  object  
 12  group_year        21402 non-null  category
dtypes: category(1), int64(6), object(6)
memory usage: 2.1+ MB


**Вывод**

Тип данных в столбцах заменен на целочисленный. Теперь мы можем приступать к обработке дубликатов.

### Обработка дубликатов  <a id="2.3"></a> 

Дубликаты могли появиться в результате неправильного ввода данных, случайного копирования данных и т.д.

Найдем количество "явных" дубликатов.

In [27]:
print(f'Количество дубликатов: {data.duplicated().sum()}')

Количество дубликатов: 54


Но если посмотреть на таблицу, а именно на столбцы, содержащие строки (str), можно заметить, что данные записаны разным регистром, и возможно являются "скрытыми" дубликатами. Приведем все символы к нижнему регистру.

In [28]:
data['education']=data['education'].str.lower()
data['family_status']=data['family_status'].str.lower()
data['gender']=data['gender'].str.lower()
data['income_type']=data['income_type'].str.lower()
data['purpose']=data['purpose'].str.lower()

In [29]:
print(f'Количество дубликатов: {data.duplicated().sum()}')

Количество дубликатов: 71


Таким образом мы выявили еще 17 скрытых дубликатов.

In [30]:
data=data.drop_duplicates().reset_index(drop=True)

**Вывод**

Мы отработали методы поиска дубликатов, в частности ручной поиск дубликатов. Теперь можно переходить к Лемматизации.

### Лемматизация <a id="2.4"></a> 

In [31]:
from pymystem3 import Mystem # импортируем библиотеку для лемматизации
m = Mystem()

def lemmatize_purpose(string):# создаем функцию для лемматизации отдельной строки
    lemma = m.lemmatize(string)
    return lemma

data['purpose'] = data['purpose'].apply(lemmatize_purpose)

dest_str = ['жилье', 'автомобиль', 'образование', 'недвижимость', 'свадьба']

def filter_purpose(srt_lem):
    """
    создаем функцию, которая отсеивает ненужные слова, возвращая только основные понятия
    """
    for filtr in dest_str:       # перебераем с помощью переменной filtr каждое понятие
        if filtr in srt_lem:     # сравниваем со строкой из таблицы
            return filtr         # в случае успеха функция возвращает основное понятие

data['purpose'] = data['purpose'].apply(filter_purpose)

In [32]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_year
0,1,06.02.1993,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,"(39.0, 47.0]"
1,1,07.01.1981,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,"(31.0, 39.0]"
2,0,25.05.1985,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,"(31.0, 39.0]"
3,3,17.04.1981,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,"(31.0, 39.0]"
4,0,25.10.2008,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,"(47.0, 56.0]"


**Вывод**

Таким образом, мы выделили 5 основных целей оформления кредита: образование, свадьба, недвижимость, автомобиль, жилье. 

### Категоризация данных <a id="2.5"></a> 

Мы видим, содержание столбцов 'education' и 'education_id', а также 'family_status' и 'family_status_id' можно объединить в словари.

In [33]:
#создадим новую таблицу, включающую в себя словарь из значений столбцов 'education' и 'education_id'
education_name=data[['education','education_id']]
#Удалим дубликаты в таблице и распечатаем её в порядке возрастания значений столбца education_id.
print(education_name.drop_duplicates().reset_index(drop=True).sort_values(by='education_id'))

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


Аналогично составим словарь по данным столбцов 'family_status' и 'family_status_id'.

In [34]:
family_status_name=data[['family_status','family_status_id']]
print(family_status_name.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id'))

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


**Вывод**

Категории выделены, можно приступать к ответам на вопросы.

## Ответы на вопросы <a id="3"></a> 

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?** <a id="3.1"></a>

In [35]:
table1=data.groupby('children')[['debt']].agg(['sum','count']).reset_index()
display(table1)

#выделим в отдельный столбец людей с просрочками, и в отдельный - общее количество людей. 
# И так по каждой группе столбца 'children'

children  debt       
             sum  count
0        0  1063  14091
1        1   444   4808
2        2   194   2052
3        3    27    330
4        4     4     41
5        5     0      9

 На этапе предобработки мы удалили ошибки в столбце 'children', равные -1 и 20. Также у нас присутствует группа с 5-ю детьми и в этой группе вообще отсутствуют "должники". Данная группа меньше 1% от общего кол-ва, и ее мы можем также отбросить. 

In [36]:
table2=data.query('children !=5').groupby('children')[['debt']].agg(['count', 'sum', 'mean'])
# также добавим функцию mean в таблицу, которая по сути здесь посчитает нам отношение "должников" к общему количеству.
display(table2)

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561

**Вывод**

Процент людей, у которых есть дети (1,2,3,4 ребенка) и были просрочки по кредиту, выше, чем процент людей с просрочками и без детей.  Данный вывод говорит нам о том, что люди с детьми допускают просрочки чаще, чем люди, у которых нет детей. Зависимость между количеством детей и количеством просрочек существует.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?** <a id="3.2"></a>

In [37]:
#аналогично проверим вторую гипотезу
table3=data.groupby('family_status')[['debt']].agg(['sum','count','mean']).reset_index()
display(table3)


family_status debt                 
                          sum  count      mean
0              в разводе   84   1189  0.070648
1         вдовец / вдова   63    951  0.066246
2       гражданский брак  385   4134  0.093130
3        женат / замужем  927  12261  0.075606
4  не женат / не замужем  273   2796  0.097639

**Вывод**

Минимальное кол-во просрочек согласно результатам допускают люди в статусе "вдовец/вдова", максимальное - люди в статусе "не женат/ не замужем" и люди, находящиеся в гражданском браке.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?** <a id="3.3"></a>

 Разделим на 4 группы столбец о доходе, чтобы сравнить их на предмет возврата кредита в срок. 

In [38]:
data['group_total'] = pd.qcut(data['total_income'],4)
data['group_total']

0        (195831.5, 2265604.0]
1         (107507.0, 142594.0]
2         (142594.0, 195831.5]
3        (195831.5, 2265604.0]
4         (142594.0, 195831.5]
                 ...          
21326    (195831.5, 2265604.0]
21327     (142594.0, 195831.5]
21328    (20666.999, 107507.0]
21329    (195831.5, 2265604.0]
21330    (20666.999, 107507.0]
Name: group_total, Length: 21331, dtype: category
Categories (4, interval[float64]): [(20666.999, 107507.0] < (107507.0, 142594.0] < (142594.0, 195831.5] < (195831.5, 2265604.0]]

In [39]:
#отфильтруем таблицу со значениями total_income первой группы
total_income_group1= data.loc[data.loc[:,'total_income']<=107507] 
count1=total_income_group1['debt'].count()  #далее считаем общее количество людей с доходом меньше среднего и кол-во таких людей с просрочками
sum1=total_income_group1['debt'].sum()

#аналогично для второй группы
total_income_group2 = data.loc[(data.loc[:,'total_income']>107507)&(data.loc[:,'total_income']<=142594)]
count2= total_income_group2['debt'].count()
sum2=total_income_group2['debt'].sum()

#для третьей группы
total_income_group3 = data.loc[(data.loc[:,'total_income']>142594)&(data.loc[:,'total_income']<=195831.5)]
count3= total_income_group3['debt'].count()
sum3=total_income_group3['debt'].sum()

#для четвертой группы
total_income_group4 = data.loc[(data.loc[:,'total_income']>195831.5)]
count4= total_income_group4['debt'].count()
sum4=total_income_group4['debt'].sum()
#создадим датафрейм, в котором наглядно будет виден результат исследования
columns=['count','sum','results']
value=[[count1, sum1, sum1/count1],[count2, sum2, sum2/count2], [count3, sum3, sum3/count3],[count4, sum4, sum4/count4]]
finish_table = pd.DataFrame(data=value, columns = columns)
display(finish_table)


,count,sum,results
0,5333,427,0.080068
1,5450,480,0.088073
2,5215,444,0.085139
3,5333,381,0.071442


**Вывод**

Из таблицы видно, что люди с доходом первых трёх групп допускают примерно одинаковое количество просрочек. А люди с доходом выше 195 тыс. допускают меньше просрочек, чем люди с меньшим доходом.

- **Как разные цели кредита влияют на его возврат в срок?** <a id="3.4"></a>

In [40]:
try:
    table9=data.groupby('purpose')[['debt']].agg(['sum','count','mean']).reset_index()
    display(table9)
except:
    print('Вывод результата невозможен. Проверь вводимые данные')

purpose debt                
                 sum count      mean
0    автомобиль  400  4279  0.093480
1         жилье  308  4437  0.069416
2  недвижимость  472  6314  0.074755
3   образование  369  3988  0.092528
4       свадьба  183  2313  0.079118

**Вывод**

Из результата можно сделать следующие выводы: Наибольшее количество просрочек допускается по кредитам на приобретение автомобиля и на образование. Меньше всего просрочек по кредитам на приобретение жилья.

## Общий вывод <a id="4"></a>

In [41]:
#создадим сводную таблицу из данных датафрейма
result=data.query('children !=-1 and children!=20 and children!=5 and gender!="XNA"').pivot_table(
    index = ['family_status','children'], 
    columns = 'purpose',
    values = 'debt', 
    aggfunc = 'sum')
result

purpose                         автомобиль  жилье  недвижимость  образование  \
family_status         children                                                 
в разводе             0               13.0   13.0          16.0         13.0   
                      1                6.0    6.0           7.0          2.0   
                      2                2.0    0.0           3.0          2.0   
                      3                0.0    0.0           1.0          NaN   
                      4                0.0    NaN           NaN          NaN   
вдовец / вдова        0               19.0    8.0          14.0         12.0   
                      1                0.0    2.0           2.0          3.0   
                      2                1.0    2.0           0.0          0.0   
                      3                NaN    0.0           0.0          0.0   
                      4                NaN    0.0           NaN          NaN   
гражданский брак      0               28.0   18.0          33.0         35.0   
                      1               17.0   14.0          18.0         18.0   
                      2                4.0    4.0           4.0          6.0   
                      3                2.0    0.0           0.0          1.0   
                      4                0.0    0.0           0.0          NaN   
женат / замужем       0              119.0  114.0         165.0        118.0   
                      1               67.0   42.0          80.0         57.0   
                      2               36.0   29.0          43.0         37.0   
                      3                3.0    6.0           5.0          3.0   
                      4                1.0    1.0           1.0          0.0   
не женат / не замужем 0               64.0   39.0          56.0         51.0   
                      1               13.0    8.0          21.0         10.0   
                      2                5.0    1.0           2.0          1.0   
                      3                0.0    NaN           1.0          0.0   
                      4                NaN    1.0           NaN          0.0   

purpose                         свадьба  
family_status         children           
в разводе             0             NaN  
                      1             NaN  
                      2             NaN  
                      3             NaN  
                      4             NaN  
вдовец / вдова        0             NaN  
                      1             NaN  
                      2             NaN  
                      3             NaN  
                      4             NaN  
гражданский брак      0           115.0  
                      1            51.0  
                      2            12.0  
                      3             5.0  
                      4             0.0  
женат / замужем       0             NaN  
                      1             NaN  
                      2             NaN  
                      3             NaN  
                      4             NaN  
не женат / не замужем 0             NaN  
                      1             NaN  
                      2             NaN  
                      3             NaN  
                      4             NaN

Из сводной таблицы ниже видно, например, что количество  замужних людей с двумя детьми, оформивших кредит на автомобиль и допустивших просрочку -  36 человек от общего числа в этой категории. В то же время людей с 2 детьми и в разводе, оформивших кредит на автомобиль и допустивших просрочку - всего двое и т.п.


Но данная информация не наглядна для общего вывода, более разумно собрать данные из таблиц нашего исследования:

In [42]:
display(table2)
display(table3)
display(table9)
display(finish_table)

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561

family_status debt                 
                          sum  count      mean
0              в разводе   84   1189  0.070648
1         вдовец / вдова   63    951  0.066246
2       гражданский брак  385   4134  0.093130
3        женат / замужем  927  12261  0.075606
4  не женат / не замужем  273   2796  0.097639

purpose debt                
                 sum count      mean
0    автомобиль  400  4279  0.093480
1         жилье  308  4437  0.069416
2  недвижимость  472  6314  0.074755
3   образование  369  3988  0.092528
4       свадьба  183  2313  0.079118

,count,sum,results
0,5333,427,0.080068
1,5450,480,0.088073
2,5215,444,0.085139
3,5333,381,0.071442


**Заключение:** Исходя из результатов таблицы можно сделать вывод, что вероятнее всего просрочку допустит человек с доходом ниже 195 тысячи рублей, семейное положение которого: "не женат\не замужем" с детьми и целью кредита является приобретение автомобиля или образование.